In [1]:
import cirq
import numpy as np
import pickle
import json
import os
from collections import Counter
from sklearn.metrics import mean_squared_error

#define utility functions

def simulate(circuit: cirq.Circuit) -> dict:
    """This funcion simulate a cirq circuit (without measurement) and output results in the format of histogram.
    """
    simulator = cirq.Simulator()
    result = simulator.simulate(circuit)
    
    state_vector=result.final_state_vector
    
    histogram = dict()
    for i in range(len(state_vector)):
        population = abs(state_vector[i]) ** 2
        if population > 1e-9:
            histogram[i] = population
    
    return histogram


def histogram_to_category(histogram):
    """This function take a histogram representations of circuit execution results, and process into labels as described in 
    the problem description."""
    assert abs(sum(histogram.values())-1)<1e-8
    positive=0
    for key in histogram.keys():
        digits = bin(int(key))[2:].zfill(20)
        if digits[-1]=='0':
            positive+=histogram[key]
        
    return positive

def count_gates(circuit: cirq.Circuit):
    """Returns the number of 1-qubit gates, number of 2-qubit gates, number of 3-qubit gates...."""
    counter=Counter([len(op.qubits) for op in circuit.all_operations()])
    
    #feel free to comment out the following two lines. But make sure you don't have k-qubit gates in your circuit
    #for k>2
    for i in range(2,20):
        assert counter[i]==0
        
    return counter

def image_mse(image1,image2):
    # Using sklearns mean squared error:
    # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html
    return mean_squared_error(image1, image2)


In [3]:
#load the actual hackthon data (fashion-mnist)
images=np.load('data/images.npy')
labels=np.load('data/labels.npy')

In [4]:
from typing import List

def to_bit(i: int) -> str: return format(i, '#010b')

flatten_images = np.zeros([2000, 28*16])
for j in range(2000):
    flatten_images[j] = images[j][:, 6:22].flatten()

image = flatten_images[0]  # Example 1 image

mean = np.mean(image)
image[np.where(image > mean)] = 1
image[np.where(image <= mean)] = 0

index_list: List[str] = []
for i, pixel in enumerate(image):
    if pixel == 1.0:
        index_list.append(to_bit(i))

# # To be our unit test
# index_list = [
#     '0b100110',
#     '0b110100',
#     '0b011010',
#     '0b001000',
#     '0b101101'
# ]
# # Expected:
# NConditionalGate(control_idx=[], control_state=[], target_idx=0, gate_type=<GateType.h: 'h'>)
# NConditionalGate(control_idx=[0], control_state=[1], target_idx=1, gate_type=<GateType.h: 'h'>)
# NConditionalGate(control_idx=[], control_state=[], target_idx=2, gate_type=<GateType.i: 'i'>)
# NConditionalGate(control_idx=[0, 1, 2], control_state=[1, 1, 0], target_idx=3, gate_type=<GateType.x: 'x'>)
# NConditionalGate(control_idx=[], control_state=[], target_idx=4, gate_type=<GateType.i: 'i'>)
# NConditionalGate(control_idx=[], control_state=[], target_idx=5, gate_type=<GateType.i: 'i'>)
# NConditionalGate(control_idx=[0, 1, 2], control_state=[1, 0, 1], target_idx=3, gate_type=<GateType.x: 'x'>)
# NConditionalGate(control_idx=[], control_state=[], target_idx=4, gate_type=<GateType.i: 'i'>)
# NConditionalGate(control_idx=[0, 1, 2, 3, 4], control_state=[1, 0, 1, 1, 0], target_idx=5, gate_type=<GateType.x: 'x'>)
# NConditionalGate(control_idx=[0, 1, 2], control_state=[1, 0, 0], target_idx=3, gate_type=<GateType.x: 'x'>)
# NConditionalGate(control_idx=[0, 1, 2, 3], control_state=[1, 0, 0, 1], target_idx=4, gate_type=<GateType.x: 'x'>)
# NConditionalGate(control_idx=[], control_state=[], target_idx=5, gate_type=<GateType.i: 'i'>)
# NConditionalGate(control_idx=[0, 1], control_state=[0, 1], target_idx=2, gate_type=<GateType.x: 'x'>)
# NConditionalGate(control_idx=[], control_state=[], target_idx=3, gate_type=<GateType.i: 'i'>)
# NConditionalGate(control_idx=[0, 1, 2, 3], control_state=[0, 1, 1, 0], target_idx=4, gate_type=<GateType.x: 'x'>)
# NConditionalGate(control_idx=[], control_state=[], target_idx=5, gate_type=<GateType.i: 'i'>)
# NConditionalGate(control_idx=[0, 1], control_state=[0, 0], target_idx=2, gate_type=<GateType.x: 'x'>)
# NConditionalGate(control_idx=[], control_state=[], target_idx=3, gate_type=<GateType.i: 'i'>)
# NConditionalGate(control_idx=[], control_state=[], target_idx=4, gate_type=<GateType.i: 'i'>)
# NConditionalGate(control_idx=[], control_state=[], target_idx=5, gate_type=<GateType.i: 'i'>)

In [5]:
import qiskit

def conditional_gate(circuit: qiskit.QuantumCircuit = None,
                     control_states: list = None,
                     control_qubits: list = None,
                     target_qubit: int = None,
                     gate_type: str = 'x'):
    '''
    This method implements either a x or h gatess controlled by a given arbitrary number of control qubits,
    when these are in the given control_states.
    '''
    
    if gate_type == 'i':
        return circuit
    else:
        for i in range(len(control_qubits)):
            if control_states[i] == 0:
                circuit.x(control_qubits[i])
    
    # Initialise all ancillas in |0>
    ancilla_qubits = range(9,9+len(control_qubits)-1)
    circuit.initialize(0, qubits=ancilla_qubits)
    circuit.barrier()
    
    if len(control_qubits) == 0:
        circuit.h(target_qubit)
    
    # If there is only one control qubit, just do a cnot or ch
    elif len(control_qubits) == 1:
        if gate_type == 'x':
            circuit.cnot(control_qubits, target_qubit)
        elif gate_type == 'h':
            circuit.ch(control_qubits, target_qubit)
    
    # For two control qubits, toffoli or cch (to be optimised)
    elif len(control_qubits) == 2:
        if gate_type == 'x':
            circuit = toffoli(circuit, control_qubits[0], control_qubits[1], target_qubit)
            # circuit.toffoli(control_qubits[0], control_qubits[1], target_qubit)
        elif gate_type == 'h':
            circuit = toffoli(circuit, control_qubits[0], control_qubits[1], ancilla_qubits[0])
            # circuit.toffoli(control_qubits[0], control_qubits[1], ancilla_qubits[0])
            circuit.ch(ancilla_qubits[0], target_qubit)
            circuit = toffoli(circuit, control_qubits[0], control_qubits[1], ancilla_qubits[0])
            # circuit.toffoli(control_qubits[0], control_qubits[1], ancilla_qubits[0])
            
    # General case taken from Mike & Ike figure 4.10
    else:
        for i in range(len(ancilla_qubits)):
            if i == 0:
                circuit = toffoli(circuit, control_qubits[0], control_qubits[1], ancilla_qubits[0])
                # circuit.toffoli(control_qubits[0], control_qubits[1], ancilla_qubits[0])
            else:
                circuit = toffoli(circuit, control_qubits[1+i], ancilla_qubits[i-1], ancilla_qubits[i])
                # circuit.toffoli(control_qubits[1+i], ancilla_qubits[i-1], ancilla_qubits[i])
                
        if gate_type == 'x':
            circuit.cnot(ancilla_qubits[-1], target_qubit)
        elif gate_type == 'h':
            circuit.ch(ancilla_qubits[-1], target_qubit)
            
        for j in range(len(ancilla_qubits)):
            i = len(ancilla_qubits)-1-j
            if i == 0:
                circuit = toffoli(circuit, control_qubits[0], control_qubits[1], ancilla_qubits[0])
                # circuit.toffoli(control_qubits[0], control_qubits[1], ancilla_qubits[0])
            else:
                circuit = toffoli(circuit, control_qubits[1+i], ancilla_qubits[i-1], ancilla_qubits[i])
                # circuit.toffoli(control_qubits[1+i], ancilla_qubits[i-1], ancilla_qubits[i])
            
    circuit.barrier()
    for i in range(len(control_qubits)):
        if control_states[i] == 0:
            circuit.x(control_qubits[i])
    return circuit
    
def toffoli(circuit: qiskit.QuantumCircuit, c0: int, c1: int, t: int):
    circuit.h(t)
    circuit.cnot(c1,t)
    circuit.tdg(t)
    circuit.cnot(c0,t)
    circuit.t(t)
    circuit.cnot(c1,t)
    circuit.tdg(t)
    circuit.cnot(c0,t)
    circuit.tdg(c1)
    circuit.t(t)
    circuit.cnot(c0,c1)
    circuit.h(t)
    circuit.tdg(c1)
    circuit.cnot(c0,c1)
    circuit.t(c0)
    circuit.s(c1)
    
    return circuit

In [7]:
from dataclasses import dataclass
from random import randint, sample, seed
from typing import List, Union, Tuple
from enum import Enum

class GateType(Enum):
    x = 'x'
    h = 'h'
    i = 'i'

@dataclass(frozen=True)
class NConditionalGate:
    """
    Data class describing a requested n-conditional gate.
    Where control and target indices correspond to qubit indices in a circuit.
    Where gate type should be 'X' or 'H' (hadamard).
    """
    control_idx: List[int]  # Qubit circuit index
    control_state: List[int]  # (Qubit) States are either 0 or 1
    target_idx: int  # Qubit circuit index
    gate_type: GateType
    
    def join(self, other: 'NConditionalGate') -> List['NConditionalGate']:
        if self.control_idx == other.control_idx and self.control_state == other.control_state and self.target_idx == other.target_idx and self.gate_type == other.gate_type:
            return []
        return [self, other]


def to_bit(i: int) -> str: return format(i, '#011b')
def to_int(b: str) -> int: return int(b, 2)
def get_state(b: str, idx: int) -> int: return int(b[2 + idx])
def get_states(b: str, idxs: List[int]) -> List[int]: return [get_state(b, idx) for idx in idxs] 

@dataclass(frozen=True)
class SplitResult:
    idx: int
    zero_based: List[str]
    one_based: List[str]
    
    def get_required_gates(self) -> List[NConditionalGate]:
        """:return: Array-like of required gate instances. Enforces simple correction by cancelling Unitary gates."""
        # Data allocation
        result: List[NConditionalGate] = []
        
        for gate in self.get_naive_required_gates():
            # Guard clause first gate appended
            if len(result) == 0:
                result.append(gate)
                continue

            last_gate = result.pop(-1)
            result.extend(last_gate.join(gate))
        
        return result
            
    def get_naive_required_gates(self) -> List[NConditionalGate]:
        """:return: Array-like of required gate instances. Based on constructing qubit-state bitstrings."""
        # Guard clause, if 1-based is empty, return identity.
        if len(self.one_based) == 0:
            return [
                NConditionalGate(
                    control_idx=[],
                    control_state=[],
                    target_idx=self.idx,
                    gate_type=GateType.i,
                )
            ]
        
        control_idx: List[int] = list(range(self.idx))
        
        # Guard clause, if 0-based is empty, return conditional X gate (based on all prior qubits)
        if len(self.zero_based) == 0:
            return [
                NConditionalGate(
                    control_idx=control_idx,
                    control_state=get_states(s, control_idx),
                    target_idx=self.idx,
                    gate_type=GateType.x,
                )
                for s in self.one_based
            ]
        
        # Else, (both zero- and one-based arrays are not empty) place conditional hadamard
        return [
            NConditionalGate(
                control_idx=control_idx,
                control_state=get_states(s, control_idx),
                target_idx=self.idx,
                gate_type=GateType.h,
            )
            for s in self.zero_based + self.one_based
        ]

# Organaze based on first index
def split_bitstring(s_array: List[str], idx: int) -> SplitResult:
    return SplitResult(
        idx=idx,
        zero_based=[s for s in s_array if get_state(s, idx) == 0],
        one_based=[s for s in s_array if get_state(s, idx) == 1],
    )

def iterate_bitstring(bitstring: List[str], idx: int) -> List[SplitResult]:
    end_result: List[SplitResult] = []
    if len(bitstring) == 0 or idx >= len(bitstring[0]) - 2:
        return end_result
    
    split_result: SplitResult = split_bitstring(bitstring, idx)
    
    end_result.append(split_result)
    end_result.extend(iterate_bitstring(split_result.one_based, idx + 1))
    end_result.extend(iterate_bitstring(split_result.zero_based, idx + 1))
    
    return end_result


# --- Code testing ---
all_gates: List[NConditionalGate] = []

for split_result in iterate_bitstring(index_list, 0):
    all_gates.extend(split_result.get_required_gates())

print(len(all_gates))
for g in all_gates:
    print(g)

96
NConditionalGate(control_idx=[], control_state=[], target_idx=0, gate_type=<GateType.h: 'h'>)
NConditionalGate(control_idx=[], control_state=[], target_idx=6, gate_type=<GateType.i: 'i'>)
NConditionalGate(control_idx=[], control_state=[], target_idx=6, gate_type=<GateType.i: 'i'>)
NConditionalGate(control_idx=[0, 1, 2, 3, 4], control_state=[1, 1, 0, 1, 1], target_idx=5, gate_type=<GateType.h: 'h'>)
NConditionalGate(control_idx=[0, 1, 2, 3, 4, 5], control_state=[1, 1, 0, 1, 1, 1], target_idx=6, gate_type=<GateType.h: 'h'>)
NConditionalGate(control_idx=[0, 1, 2, 3, 4, 5, 6], control_state=[1, 1, 0, 1, 1, 1, 1], target_idx=7, gate_type=<GateType.h: 'h'>)
NConditionalGate(control_idx=[], control_state=[], target_idx=8, gate_type=<GateType.i: 'i'>)
NConditionalGate(control_idx=[0, 1, 2, 3, 4, 5, 6], control_state=[1, 1, 0, 1, 1, 0, 1], target_idx=7, gate_type=<GateType.h: 'h'>)
NConditionalGate(control_idx=[0, 1, 2, 3, 4, 5, 6], control_state=[1, 1, 0, 1, 1, 0, 0], target_idx=7, gate_typ

In [9]:
q = qiskit.QuantumRegister(16)
circuit = qiskit.QuantumCircuit(q)

for gate in all_gates:
    if gate.gate_type == GateType.i:
        circuit.i(gate.target_idx)
        continue
    circuit = conditional_gate(circuit, gate.control_state, gate.control_idx, gate.target_idx, gate.gate_type.value)

# circuit.draw()